In [2]:
import os
import csv
import pymongo
from pymongo import MongoClient

# Configuração MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Ajuste o URL do MongoDB conforme necessário
#db = client['gastospublicos']  # Nome do banco de dados

In [4]:
def convert_and_upload_to_mongodb(root_dir, mongodb_uri):
    # Establish MongoDB connection
    client = pymongo.MongoClient(mongodb_uri)
    db = client['gastospublicos']  # Replace with your actual database name

    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.csv'):
                filepath = os.path.join(subdir, file)

                # UTF-8 conversion 
 #               with open(filepath, 'r', encoding='utf-8', errors='replace') as f:
 #                   content = f.read()
 #               with open(filepath, 'w', encoding='utf-8') as f:
 #                   f.write(content)

                # Extract data from CSV
                with open(filepath, 'r', encoding='utf-8') as f:
                    reader = csv.reader(f, delimiter=';', quotechar='"') 
                    header = next(reader) 
                    reader = csv.DictReader(f, fieldnames=header, delimiter=';', quotechar='"') 
                    data = []
                    for row in reader:
                        cleaned_row = {str(k) if k is not None else 'unknown_column': v for k, v in row.items()}
                        data.append(cleaned_row)

                # Create collection dynamically (using root folder name)
                collection_name = os.path.basename(root_dir)
                collection = db[collection_name]

                # Insert data into MongoDB
                collection.insert_many(data)

if __name__ == "__main__":
    starting_directory = input("Enter the starting directory: ")
    mongodb_uri = input("Enter your MongoDB connection URI: ") 
    convert_and_upload_to_mongodb(starting_directory, mongodb_uri)
    print("Conversion and upload complete!")

Conversion and upload complete!
